In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [5]:
dataset = 'onion' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [1, 5, 10, 20]
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [6]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [8]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

(30, 13)
(30, 18)
(30, 23)


,model,modality,date,Recall@1,Recall@5,Recall@10,nDCG@1,nDCG@5,nDCG@10
22,LATTICE,textual,2025_01_15_00_39_49,0.000000,0.000227,0.000582,0.000000,0.000159,0.000331
3,Random,None,2025_01_14_16_21_46,0.000128,0.000409,0.000811,0.000293,0.000379,0.000546
2,MostPop,None,2025_01_14_16_21_46,0.003608,0.011235,0.019006,0.010782,0.011125,0.013836
23,NeuMF,None,2025_01_15_01_27_09,0.003813,0.012371,0.018402,0.011222,0.011878,0.013911
24,MMGCN,textual,2025_01_15_01_33_55,0.013072,0.046306,0.072034,0.034473,0.041808,0.051582
15,MMGCN,audio,2025_01_14_22_16_32,0.013262,0.048216,0.075262,0.036453,0.043686,0.053750
7,MMGCN,emotion,2025_01_14_17_34_10,0.015008,0.049347,0.072906,0.039900,0.045638,0.054493
8,GRCN,emotion,2025_01_14_17_47_08,0.040274,0.126900,0.176205,0.120508,0.124491,0.141616
4,BM3,emotion,2025_01_14_16_44_48,0.042990,0.141013,0.197274,0.122268,0.134118,0.153961
19,BM3,textual,2025_01_14_23_50_55,0.043789,0.142446,0.199090,0.124615,0.134862,0.155051


In [9]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [10]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)